# DenseNets

In [1]:
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'../data generation')
sys.path.insert(0,'../model architecture')

In [2]:
from data_generator import load_dataset
from models import *
from experiment_tools import *
from params import *

Using backend: pytorch


## Dataset
### Set hyper-parameters: dataset

In [3]:
test_cases, dataset_size, std_scaler, test_type = get_test_params()

### Select a dataset

In [4]:
idx = 0
test_case = test_cases[idx] 
test_case

'pglib_opf_case24_ieee_rts.m'

### Load a dataset

In [5]:
dataset = load_dataset(test_case=test_case,
                       dataset_size=dataset_size,
                       std_scaler=std_scaler,
                       test_type=test_type,
                       gen_only=True)

### Check the shape of the dataset
- number of samples
- number of buses
- number of generators

In [6]:
# (num_samples, num_buses),(num_samples, num_gens) 
dataset['x'].shape, dataset['y'].shape 

((300, 24, 1), (300, 24, 6))

## Model
### Set hyper-parameters: model

In [7]:
model_name = "dense_net"
num_epochs, batch_size, lr = get_h_params(model_name)

### Set data-loaders

In [8]:
train_loader, val_loader, test_loader = data_prep(dataset=dataset,
                                                  batch_size=batch_size)

/Users/jeehyunpark/Library/Mobile Documents/com~apple~CloudDocs/jhyun/UT/EE 394V - DATA ANALYTICS POWER SYS/Project_EE394V_SPR2021/code/experiments/experiment_tools.py:31: RuntimeWarning: invalid value encountered in true_divide
  x_normed = np.nan_to_num(dataset["x"] / dataset["x"].max(axis=0), nan=0.0)


### Check the shape of the data-loaders
- number of batch
- number of buses
- number of generators

In [9]:
feature, label = next(iter(train_loader))
feature.shape, label.shape

(torch.Size([32, 24, 1]), torch.Size([32, 24, 6]))

In [10]:
feature, label = next(iter(val_loader))
feature.shape, label.shape

(torch.Size([32, 24, 1]), torch.Size([32, 24, 6]))

In [11]:
feature, label = next(iter(test_loader))
feature.shape, label.shape

(torch.Size([1, 24, 1]), torch.Size([1, 24, 6]))

### Set a model

In [12]:
# model
dense_net = DenseNet(dataset['x'].shape[2], dataset['y'].shape[2], dataset['y'].shape[1])

# loss object
loss_func = nn.MSELoss()

# optim
optimizer = optim.SGD(dense_net.parameters(), lr=lr, momentum=0.9)

dense_net.eval()

DenseNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dense_1): Linear(in_features=24, out_features=512, bias=True)
  (dense_2): Linear(in_features=512, out_features=512, bias=True)
  (dense_3): Linear(in_features=512, out_features=144, bias=True)
  (tanh): Tanh()
)

### Train the model

In [13]:
train(net=dense_net,
      train_loader=train_loader,
      val_loader=val_loader,
      optimizer=optimizer,
      loss_func=loss_func,
      model_name=model_name,
      dataset_name=test_case,
      num_epochs=num_epochs)

> case: pglib_opf_case24_ieee_rts
> model: dense_net
- num of params: 349328
- training
	- validation loss decreased (inf->0.19840): the best model was updated.
	- validation loss decreased (0.19840->0.19303): the best model was updated.


### Test the model

In [14]:
test(net=dense_net,
     test_loader=test_loader,
     model_name=model_name,
     dataset_name=test_case)

- testing
	accuracy: 489.5833%
